In [ ]:
import os
from __init__ import *

__ORIG_WD__ = os.getcwd()
os.chdir(f"{__ORIG_WD__}/../data_collectors")

from covid19_genome import Covid19Genome

os.chdir(__ORIG_WD__)


In [ ]:
dc = Covid19Genome()

In [ ]:
lineages = dc.getLocalLineages()

mappings = []
accessions = for lineage in lineages:
    for accession in dc.getLocalAccessionsPaths(lineage):
        mappings.append((accession, lineage))


In [ ]:
import tensorflow as tf

def compute_coverage(n, indexes, read_lengths):
    # Create a range tensor [0, 1, 2, ... n-1]
    range_tensor = tf.range(n, dtype=tf.int32)

    # Expand dims for broadcasting
    expanded_range = tf.expand_dims(range_tensor, 0)
    expanded_indexes = tf.expand_dims(indexes, 1)
    expanded_lengths = tf.expand_dims(read_lengths, 1)

    # Create a binary mask for each read
    # Each row in the mask represents where the read starts and its length
    print((expanded_range >= expanded_indexes).shape)
    mask = tf.logical_and(expanded_range >= expanded_indexes,
                          expanded_range < expanded_indexes + expanded_lengths)

    # Convert the boolean mask to integers and sum across rows
    coverage = tf.reduce_sum(tf.cast(mask, tf.int32), axis=0)
    
    return coverage

# Example
n = 10
indexes = tf.constant([1, 2, 5], dtype=tf.int32)
read_lengths = tf.constant([3, 20, 2], dtype=tf.int32)

coverage_per_base = compute_coverage(n, indexes, read_lengths)
print(coverage_per_base.numpy())  # Expected: [0, 1, 1, 1, 0, 1, 1, 0, 0, 0]


In [ ]:
tf.tensor_scatter_nd_add(
    tf.zeros([3], tf.constant([0], [0]), tf.constant([1,1]))
)

In [ ]:
import tensorflow as tf

def propagate_values(vector):
    # Find where the vector is not zero
    non_zeros = tf.cast(tf.not_equal(vector, 0), dtype=tf.int32)

    # Get the cumulative sum of non-zero locations
    cum_sum = tf.cumsum(non_zeros)

    # Create a mask to gather elements
    mask = cum_sum * non_zeros

    # Use tf.gather to replace zeros with latest non-zero value
    propagated_values = tf.gather(tf.boolean_mask(vector, mask), cum_sum - 1)

    return propagated_values

# Example
vec = tf.constant([0, 0, 0, -1, 0, 0, 0, -2, 0, 0, 0], dtype=tf.int32)
result = propagate_values(vec)
print(result.numpy())  # Expected: [0,0,0,3,3,3,3,3,8,8,8]


In [ ]:
a = tf.constant([1,2,3,4,5])
tf.diff(a)

In [ ]:
def random_partition(balls, bins):
    """
    1 represents a ball
    0 represents a partition
    """
    # randomize the ones indexes
    balls_indexes = tf.random.shuffle(tf.range(balls+bins-1))[:balls]
    balls_and_partitions = tf.concat([
        tf.scatter_nd(
            indices=tf.expand_dims(balls_indexes, 1),
            updates=tf.ones(balls, dtype=tf.dtypes.int32),
            shape=[bins+balls-1]
        ), 
        [0]
    ], axis=-1)
    
    # cumsum of balls and partitions
    cumsum = tf.cumsum(balls_and_partitions)

    summed_balls_per_partition = tf.boolean_mask(cumsum, balls_and_partitions == 0)

    prev_balls_per_partition = tf.concat([[0], summed_balls_per_partition[:-1]], axis = -1)

    return summed_balls_per_partition - prev_balls_per_partition

In [ ]:
def _compute_coverage_per_base(genome_length, read_length, coverage):
    num_reads = tf.cast(tf.math.ceil(coverage * genome_length / read_length), tf.int32)
    read_starts_per_base = tf.concat(
        [
            random_partition(
                balls = num_reads,
                bins = genome_length - read_length + 1,
            ),
            tf.zeros(read_length, dtype=tf.int32)
        ],
        axis=-1
    )
    tf.print(read_starts_per_base)
    read_ends_per_base = -1*tf.roll(read_starts_per_base, read_length, axis=0)
    tf.print(read_ends_per_base)
    coverage_per_base = tf.cumsum(read_starts_per_base + read_ends_per_base)
    return coverage_per_base

In [ ]:
_compute_coverage_per_base(5, 2, 5)

In [5]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# The inputs are 128-length vectors with 10 timesteps, and the
# batch size is 4.

conv = tf.keras.layers.Conv1D(32, 3, activation='relu', padding="same")

input_shape = (32, 256, 128, 4)
x = tf.random.normal(input_shape)
y = (x)
print(y.shape)
y = tf.nn.max_pool(
    y,
    ksize=[1, 1, 2, 1],  # Pooling size: No pooling on batch1, batch2, and the last dimension. Pool every 2 elements in the third dimension.
    strides=[1, 1, 2, 1],  # Stride: Move by 2 elements in the third dimension.
    padding='VALID'
)
print(y.shape)


(32, 256, 128, 32)
(32, 256, 64, 32)


In [12]:
# as first layer in a Sequential model

reshape = tf.keras.layers.Reshape((None, 3, 4))
# model.output_shape == (None, 3, 4), `None` is the batch size.


In [15]:
x = tf.random.normal((None,10,3,4))
# print(x)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.